In [ ]:
from pyspark.sql import SparkSession
from operator import add
import time
import re
from pyspark.sql.functions import flatten,udf, col, explode
from pyspark.sql.types import ArrayType, StringType
import csv

def genderTags(lst):
    genderList = []
    for element in lst:
        if re.search("female|Female", element) and "female" not in genderList:
            genderList.append("female")
            continue
        elif re.search("male|Male", element) and "male" not in genderList:
            genderList.append("male")
            continue
        
    return genderList

def removeGenderTags(lst):
    tags = []
    for element in lst:
        if re.search("female|Female", element) or re.search("male|Male", element):
            continue
        else:
            tags.append(element)
        
        
    return tags

root ='hdfs://192.168.2.200:9000/user/ubuntu/lastfm_train/'
letters = ['A','B','C','E','G','J','K','L','M','N','O','P','Q','R','S','T','U','W','X','Y','Z']
paths=[]
timedict = {}
for i in letters:
    paths.append(root+i)


for i in range(len(letters)):

        spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.218:7077") \
        .appName("Scalingtest")\
        .config("spark.executor.cores",2)\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", False)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.driver.port",9998)\
        .config("spark.blockManager.port",10005)\
        .config("spark.executor.memory","2g")\
        .getOrCreate()


        spark_context = spark_session.sparkContext

        spark_context.setLogLevel("ERROR")

        start = time.time()
        data = spark_session.read.option("recursiveFileLookup", "true").json(paths[i]).cache()

        dataframe = data.withColumn("New tags", flatten(data.tags))

        removeNumbers = udf(lambda lst: lst[0::2], ArrayType(StringType()))

        dataFrame = dataframe.withColumn("Only tags", removeNumbers(col("New tags")))

        genderTagUDF = udf(genderTags, ArrayType(StringType()))

        # Create a new column representing wether a song has been tagged with "female", "male" or both
        dataframe = dataFrame.withColumn("Gender tag", genderTagUDF(col("Only tags")))

        removeGenderTagUDF = udf(removeGenderTags, ArrayType(StringType()))

        # Create a new column where the"female" and "male" tags have been removed from the other tags
        filteredDF = dataframe.withColumn("Tags", removeGenderTagUDF(col("Only tags")))

        genderDataFrame = filteredDF.select("Tags", "Gender tag")

        genderDataFrame = genderDataFrame.select(genderDataFrame["Gender tag"], explode(genderDataFrame["Tags"]))
        genderDataFrame = genderDataFrame.withColumnRenamed("col", "Tag")

        genderDataFrame = genderDataFrame.select(explode(genderDataFrame["Gender tag"]), genderDataFrame["Tag"])
        genderDataFrame = genderDataFrame.withColumnRenamed("col", "Gender")

        tagsFemale = genderDataFrame.select("Tag").filter(genderDataFrame["Gender"] == "female").groupBy("Tag").agg({"Tag": "count"})\
        .withColumnRenamed("count(Tag)","Frequency").orderBy("Frequency", ascending=False)

        print(tagsFemale.show())
        tagsMale = genderDataFrame.select("Tag").filter(genderDataFrame["Gender"] == "male").groupBy("Tag").agg({"Tag": "count"})\
        .withColumnRenamed("count(Tag)","Frequency").orderBy("Frequency", ascending=False)

        print(tagsMale.show())
        timeend = (time.time()-start)/60
        timedict[letters[i]] = timeend
        print()
        print(f'execution time in minutes: {timeend}')

        tagsFemale.toPandas().to_csv(f"/home/ubuntu/Project/Urlich/results/4workers/{letters[i]}_female.csv")
        tagsMale.toPandas().to_csv(f"/home/ubuntu/Project/Urlich/results/4workers/{letters[i]}_male.csv")

        spark_session.stop()


with open('/home/ubuntu/Project/Urlich/results/4workers/timeres.csv', 'w') as csv_file:
        csv_write = csv.writer(csv_file)
        for key, value in timedict.items():
                csv_write.writerow([key, value])





